<h1 align="center">
🧹 Limpeza Catálogo de Teses e Dissertações <br/>
<img src="https://dadosabertos.capes.gov.br/img/caixa.png"  alt="Dados Capes"/>
</h1>

Em nosso conjunto de dados possuímos informações sobre teses e dissertações defendidas no período de 1987-2023. No notebook de [download de metadados](notebooks/1.obter_catalogos.ipynb), realizamos o download e junção de todos os conjuntos.

Desejamos extrair informações e estatísticas sobre o conteúdo das teses e dissertações, para isso utilizamos a coluna `resumo`. 

Com base no resumo, gostaríamos de responder algumas perguntas sobre os dados, tais como:

- quais são os assuntos mais presentes em uma grande área de conhecimento? E em uma área de conhecimento?
- quais são os assuntos mais abordados em cada período de tempo?
- os assuntos desses trabalhos são diferentes em cada região? E em nível de estadual ou nível de universidade?

Entretanto para que isso seja possível, precisamos verificar se não houve erro de preenchimento ou valores faltantes. Assim realizaremos uma limpeza de dados nas colunas referentes a essas informações.



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.auto import tqdm

tqdm.pandas()

## Seleção das colunas de interesse

In [ ]:
workdir = Path("../..")
df = pd.read_parquet(workdir / "data" / "catalogo-de-teses-e-dissertacoes.parquet")

Dentre as colunas de interesse selecionamos:

- ano_base
- resumo
- nome_grande_area
- nome_area_conhecimento
- regiao
- sigla_uf
- sigla_entidade_ensino

In [ ]:
df = df.loc[
    :,
    [
        "AN_BASE",
        "DS_RESUMO",
        "NM_PRODUCAO",
        "NM_GRANDE_AREA_CONHECIMENTO",
        "NM_AREA_CONHECIMENTO",
        "NM_REGIAO",
        "SG_UF_IES",
        "SG_ENTIDADE_ENSINO",
    ],
]

df.head()

In [ ]:
df.info()

Podemos perceber que há palavras não acentuadas nos textos de 1987. Também notamos que há valores não preenchidos na coluna de resumo, como a nossa análise a considera, desconsideramos as linhas onde `resumo` é nulo.

In [ ]:
df = df.dropna(subset=["DS_RESUMO"])
df.info()

Por fim removemos as linhas duplicadas cadastradas em nosso *dataset*.



In [ ]:
df = df.drop_duplicates()
df.info()

## Limpeza do Resumo

Analisamos nosso dataset em busca de textos sem valor para nossa análise buscando por resumos com menos de 60 caracteres. Mas antes disso realizamos uma padronização nos resumos, deixando-os em caixa baixa, removendo números e pontuações.

In [ ]:
# Criamos uma coluna adicional com o propósito de facilitar a filtragem de resumos não relevantes
df["DS_RESUMO_CLEANED"] = (
    df["DS_RESUMO"]
    .str.lower()
    .str.replace(r"[^\w\s]+", " ", regex=True)  # mantém apenas letras e números
    .str.replace(r"\d+", " ", regex=True)  # remove números
    .str.replace(r"\s+", " ", regex=True)  # remove excesso de espaços
    .str.strip()
)

In [ ]:
for index, _ in (
    df[df["DS_RESUMO_CLEANED"].str.len() < 60]["DS_RESUMO_CLEANED"]
    .value_counts()
    .items()
):
    print(index)

Podemos notar que a maioria das sentenças com até 60 caracteres há informações sobre o status da entrega da dissertação, sequências de letras ou palavras sem sentido como "iiiiiiiiiiiiii", "hnjkm l ç". Portanto, optamos por remover colunas com menos de 60 caracteres.

In [ ]:
df = df[df["DS_RESUMO_CLEANED"].str.len() > 60]
df.info()

Usamos como base a norma da ABNT  [ABNT NBR 6028](http://plone.ufpb.br/secretariado/contents/documentos/2021_ABNT6028Resumo.pdf) para contar as palavras do resumo. 


**4.1.8 Quanto à sua extensão, convém que os resumos tenham:**
1. 150 a 500 palavras nos trabalhos acadêmicos e relatórios técnicos e/ou científicos;
2. 100 a 250 palavras nos artigos de periódicos;
3. 50 a 100 palavras nos documentos não contemplados nas alíneas anteriores.


In [ ]:
df["WC"] = df["DS_RESUMO_CLEANED"].str.split().str.len()

In [ ]:
df["WC"].describe(percentiles=[0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99]).round(2)

In [ ]:
ax = df["WC"].plot.hist(bins=100, alpha=0.5)
ax.set_xlabel("Número de palavras")
ax.set_ylabel("Número de resumos")
ax.set_title("Histograma do número de palavras nos resumos")
ax.grid()
plt.show()

Menos de 1% dos resumos tem até 46 palavras, abaixo selecionamos os resumos com até 25 palavras para análise.

In [ ]:
# listar resumos com menos de 10 palavras
for index, row in df[df["WC"] < 25]["DS_RESUMO_CLEANED"].items():
    print(index, row)

Vemos que resumos com até 25 palavras são semelhantes a títulos de trabalhos, como "análise ergonômica da tarefa como instrumento de reprojeto de atividades para gestão do conhecimento um estudo de caso"  ou "abelhas africanizadas na cidade de são paulo uma abirdagem epidemiológica". Como a norma da ABNT sugere que resumos tenham entre 150 a 500 palavras, optamos por remover os resumos com até 25 palavras.

In [ ]:
# remover resumos com menos de 10 palavras
df = df[df["WC"] >= 25]

df.shape

Por fim, realizamos a contagem de palavras que consideramos válidas em cada resumo, para isso usamos embeddings de palavras em português.

In [ ]:
%%bash

wget -c http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s100.zip -O cbow_s100.zip
unzip cbow_s100.zip -d embeddings

In [ ]:
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format(
    "embeddings/cbow_s100.txt", binary=False
)

In [ ]:
def count_oov(text):
    return sum(1 for word in text if word not in model)


df["OOV"] = df["DS_RESUMO_CLEANED"].str.lower().str.split().progress_apply(count_oov)
df["OOV_RATE"] = df["OOV"] / df["WC"]

In [ ]:
df["OOV_RATE"].describe(percentiles=[0.9, 0.99, 0.999, 0.9999]).round(2)

In [ ]:
for index, row in df[df["OOV_RATE"] > 0.35]["DS_RESUMO_CLEANED"].items():
    print(index, row)
    print()
    print("--" * 50)

In [ ]:
df = df[df["OOV_RATE"] <= 0.35]
df.shape

Por fim, removemos as linhas onde o resumo se repete.



In [ ]:
df.columns

In [ ]:
df = df.drop_duplicates(subset=["DS_RESUMO", "NM_PRODUCAO"])
df.info()

## Limpeza nas demais colunas

Primeiro verificamos se há inconsistência nas demais colunas do dataset.

In [ ]:
df["AN_BASE"].unique()

In [ ]:
df["NM_GRANDE_AREA_CONHECIMENTO"].unique()

In [ ]:
df["NM_AREA_CONHECIMENTO"].unique()

In [ ]:
df["NM_REGIAO"].unique()

In [ ]:
df["SG_UF_IES"].unique()

In [ ]:
df["SG_ENTIDADE_ENSINO"].unique()

## Salvando dados
Como não encontramos inconsistências nas demais colunas finalizamos a nossa análise e portanto salvamos o novo conjunto de dados.

In [ ]:
df.to_parquet(workdir / "data" / "catalogo-de-teses-e-dissertacoes-limpo.parquet")